In [1]:
# =====================================================
# FAKE NEWS DETECTION – RoBERTa-base + GonzaloA/fake_news
# =====================================================

# 1. CÀI ĐẶT & IMPORT
# !pip install -q transformers datasets torch scikit-learn pandas numpy psutil accelerate

import os, re, shutil, psutil, warnings
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset, Dataset, DatasetDict
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight
from transformers import (
    RobertaTokenizerFast,               # <--- Tokenizer cho RoBERTa
    RobertaForSequenceClassification,   # <--- Model RoBERTa
    Trainer, TrainingArguments, EarlyStoppingCallback,
    DataCollatorWithPadding
)
from google.colab import drive

warnings.filterwarnings("ignore")

# 2. KIỂM TRA GPU
if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    print(f"Device: {device_name} | CUDA: True")
else:
    print("Device: CPU")

# 3. MOUNT DRIVE
drive.mount('/content/drive', force_remount=False)
OUTPUT_DIR = "/content/drive/MyDrive/GonzaloA_RoBERTa_base_Pro" # <--- Tên thư mục mới
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Hàm quản lý checkpoint
def manage_checkpoints(output_dir, keep_latest=2):
    ckpts = [c for c in os.listdir(output_dir) if c.startswith("checkpoint-")]
    if len(ckpts) <= keep_latest: return
    def idx_from_name(n):
        parts = n.split("-")
        try: return int(parts[-1])
        except: return os.path.getmtime(os.path.join(output_dir, n))
    ckpts_sorted = sorted(ckpts, key=idx_from_name)
    for old_ckpt in ckpts_sorted[:-keep_latest]:
        shutil.rmtree(os.path.join(output_dir, old_ckpt), ignore_errors=True)

def get_last_checkpoint(output_dir):
    ckpts = [c for c in os.listdir(output_dir) if c.startswith("checkpoint-")]
    if not ckpts: return None
    def idx_from_name(n):
        try: return int(n.split("-")[-1])
        except: return 0
    ckpts_sorted = sorted(ckpts, key=idx_from_name, reverse=True)
    return os.path.join(output_dir, ckpts_sorted[0])

last_checkpoint = get_last_checkpoint(OUTPUT_DIR)
print("Checkpoint gần nhất:", last_checkpoint or "Không có → Train từ đầu")

# 4. TẢI DATASET GonzaloA/fake_news
print("\nĐang tải dataset GonzaloA/fake_news...")
dataset = load_dataset("GonzaloA/fake_news")

# 5. XỬ LÝ DỮ LIỆU
def process_split(ds_split):
    df = pd.DataFrame(ds_split)
    # Ghép title + text
    df['content'] = df.get('title', '').fillna('') + " [SEP] " + df.get('text', '').fillna('')
    return df

if 'validation' in dataset and 'test' in dataset:
    print("Sử dụng các tập train/val/test có sẵn.")
    train_df = process_split(dataset['train'])
    val_df = process_split(dataset['validation'])
    test_df = process_split(dataset['test'])
else:
    print("Tự chia tập dữ liệu...")
    df = process_split(dataset['train'])
    from sklearn.model_selection import train_test_split
    train_df, temp_df = train_test_split(df, test_size=0.25, random_state=42, stratify=df['label'])
    val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

# Hàm làm sạch văn bản
def clean_text(text):
    if not isinstance(text, str): return ""
    t = text.lower()
    t = re.sub(r'https?://\S+|www\.\S+', ' ', t)
    t = re.sub(r'<.*?>', ' ', t)
    t = re.sub(r'[^a-zA-Z0-9\s]', ' ', t)
    return re.sub(r'\s+', ' ', t).strip()

print("Đang làm sạch văn bản...")
for df in [train_df, val_df, test_df]:
    df['content'] = df['content'].apply(clean_text)
    df.drop(df[df['content'].str.len() < 20].index, inplace=True)

print(f"Sizes -> Train: {len(train_df):,} | Val: {len(val_df):,} | Test: {len(test_df):,}")

# 6. TÍNH CLASS WEIGHTS
classes = np.array([0, 1])
class_weights = compute_class_weight('balanced', classes=classes, y=train_df['label'])
class_weight_dict = {0: float(class_weights[0]), 1: float(class_weights[1])}
print("Class weights:", class_weight_dict)

train_dataset = Dataset.from_pandas(train_df[['content', 'label']].reset_index(drop=True))
val_dataset = Dataset.from_pandas(val_df[['content', 'label']].reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df[['content', 'label']].reset_index(drop=True))

dataset_dict = DatasetDict({"train": train_dataset, "validation": val_dataset, "test": test_dataset})

# 7. TOKENIZER RoBERTa
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

def tokenize_fn(batch):
    # RoBERTa cũng dùng max_length 512
    return tokenizer(batch["content"], truncation=True, max_length=512, padding=False)

print("Tokenizing...")
tokenized = dataset_dict.map(tokenize_fn, batched=True, batch_size=1000, remove_columns=['content'])
tokenized = tokenized.rename_column("label", "labels")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 8. MODEL RoBERTa
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
model.config.id2label = {0: "Fake", 1: "Real"}
model.config.label2id = {"Fake": 0, "Real": 1}

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,

    # RoBERTa khá nặng, cẩn thận OOM
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=1,

    warmup_ratio=0.1,
    weight_decay=0.01,
    learning_rate=2e-5,             # Standard LR

    eval_strategy="epoch",          # Đã sửa lỗi evaluation_strategy
    save_strategy="epoch",

    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_total_limit=2,
    fp16=torch.cuda.is_available(), # Bắt buộc bật fp16
    report_to="none"
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted", zero_division=0)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

class WeightedTrainer(Trainer):
    # Đã thêm fix num_items_in_batch
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        weight_tensor = torch.tensor([class_weights[0], class_weights[1]], dtype=torch.float32, device=model.device)
        loss_fct = torch.nn.CrossEntropyLoss(weight=weight_tensor)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# 9. TRAIN & EVAL
print("\nBẮT ĐẦU HUẤN LUYỆN RoBERTa-BASE...")
trainer.train(resume_from_checkpoint=last_checkpoint)
manage_checkpoints(OUTPUT_DIR)

print("\nĐÁNH GIÁ TRÊN TẬP TEST...")
results = trainer.evaluate(tokenized["test"])
print(results)

# Lưu model
final_path = os.path.join(OUTPUT_DIR, "final_best_model")
trainer.save_model(final_path)
tokenizer.save_pretrained(final_path)
print(f"Đã lưu model RoBERTa tại: {final_path}")

Device: Tesla T4 | CUDA: True
Mounted at /content/drive
Checkpoint gần nhất: Không có → Train từ đầu

Đang tải dataset GonzaloA/fake_news...


README.md: 0.00B [00:00, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


dataset_infos.json: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/38.8M [00:00<?, ?B/s]

...


BẮT ĐẦU HUẤN LUYỆN RoBERTa-BASE...


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.061000,0.052753,0.982009,0.982015,0.982009,0.982010


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.061000,0.052753,0.982009,0.982015,0.982009,0.982010
2,0.043100,0.101162,0.978681,0.978948,0.978681,0.978655
3,0.021100,0.047028,0.989402,0.989402,0.989402,0.989402



ĐÁNH GIÁ TRÊN TẬP TEST...


{'eval_loss': 0.04576059803366661, 'eval_accuracy': 0.9889121596649009, 'eval_precision': 0.9889278177420925, 'eval_recall': 0.9889121596649009, 'eval_f1': 0.9889139613950106, 'eval_runtime': 62.8624, 'eval_samples_per_second': 129.123, 'eval_steps_per_second': 4.041, 'epoch': 3.0}
Đã lưu model RoBERTa tại: /content/drive/MyDrive/GonzaloA_RoBERTa_base_Pro/final_best_model
